In [ ]:
import os
import sys

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory of `maps` (which is `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Add `src` to the module search path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import pandas as pd
import numpy as np
import folium.features
from sklearn.cluster import DBSCAN
from sklearn.cluster import HDBSCAN
from sklearn.cluster import OPTICS
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures
from pyproj import Transformer
import seaborn as sns
import networkx as nx
from scipy.spatial import Delaunay
from itertools import combinations
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import pyproj


from python_scripts.road_detection.road_utils import *
from python_scripts.miscellaneaous.data_processing import extract_data
from python_scripts.graphs.graphs_creation import delaunay_graph
from python_scripts.city.city_utils import plotMapWithColors, mean_distance_choice, mean_distance_to_NN
from python_scripts.ihm.maps.mapUtils import *
from python_scripts.neighbours_criteria.enhanced_criteria import km_distance
from python_scripts.neighbours_criteria.simple_criteria import distance_criterion, angle_criterion


In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')

In [ ]:
df_extracted = extract_data(df, provider='Orange', techno='4g')#, region='Normandie') #Occitanie, Île-de-France
G, pos = delaunay_graph(df_extracted)

In [ ]:
mean_distances = mean_distance_to_NN(df_extracted[['x', 'y']], n_neighbours=3) # 3 to have more neighbours

In [ ]:
cityStationsLabels = []

for station in mean_distances.index:
    if(mean_distances[station]<=0.5):
        cityStationsLabels.append(station)

In [ ]:
cityLabels = pd.Series(DBSCAN(eps=3000, min_samples=1).fit(df_extracted[['x','y']].loc[cityStationsLabels]).labels_, index = cityStationsLabels)
clusters = cityLabels.unique()
num_clusters = len(clusters) - (1 if -1 in clusters else 0)

In [ ]:
cityBsStationCount = cityLabels.value_counts()

# bigCitiesClusters = cityBsStationCount[cityBsStationCount >= 27].index
# littleCitiesClusters = cityBsStationCount[cityBsStationCount < 27].index


In [ ]:
cityCenters = {cityLabel : (0,0) for cityLabel in cityLabels.unique()}
for bs_id, x, y in df_extracted[['x', 'y']].loc[cityStationsLabels].itertuples():
    coords = cityCenters.get(cityLabels[bs_id])
    cityCenters[cityLabels[bs_id]] = (x + coords[0], y + coords[1])

for cluster in clusters:
    cityCenters[cluster] = cityCenters.get(cluster) / cityBsStationCount[cluster]

In [ ]:
array1 = np.array(list(cityCenters.values())).reshape(-1,2)
array2 = np.array(df_extracted[['x', 'y']].values)

# Initialize the NearestNeighbors model and fit it to list2
nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(array2)

# Find the nearest neighbors for each point in list1
distances, indices = nbrs.kneighbors(array1)

cityNames = {cityId : df_extracted['nom_com'].loc[df_extracted.index[indice][0]] for (cityId, indice) in zip(cityCenters.keys(), indices)}



In [ ]:
def rgb_to_hex(rgb):
        return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))

def labelToColor(clustId, clusters, palette):   
    if (clustId == -1):
        return 'gray'
    else:
        return rgb_to_hex(palette[list(clusters).index(clustId)- (1 if -1 in clusters else 0)])

In [ ]:
palette = sns.color_palette("hsv", num_clusters)
colors = cityLabels.apply(lambda clustId : labelToColor(clustId, clusters, palette))

In [ ]:
map = folium.Map(location=np.mean(df_extracted[['latitude','longitude']].loc[cityStationsLabels], axis=0), zoom_start=7, tiles="Cartodb Positron")
citiesLayer = folium.FeatureGroup(f"Cities").add_to(map)
cityCenterLayer = folium.FeatureGroup(f"Cities centers").add_to(map)
cityNameLayer = folium.FeatureGroup(f"Cities names").add_to(map)

for bs_id, x, y in df_extracted[['latitude', 'longitude']].loc[cityStationsLabels].itertuples():
    color = colors[bs_id]
    dot = folium.CircleMarker(location=[x, y], color=color, radius=1, popup=cityNames.get(cityLabels.get(bs_id)))
    citiesLayer.add_child(dot)


for cluster in clusters:
    centerXY = cityCenters[cluster]

    lambert93 = pyproj.CRS("EPSG:2154")  # Lambert 93
    wgs84 = pyproj.CRS("EPSG:4326")       # WGS84
    transformer = pyproj.Transformer.from_crs(lambert93, wgs84, always_xy=True)

    centerLl = transformer.transform(centerXY[0], centerXY[1])[::-1]

    dot = folium.CircleMarker(location=centerLl, color='black', fill_opacity = 1, fill=True, radius=4, popup=cityNames.get(cluster))
    cityCenterLayer.add_child(dot)


    offset = np.array([0, 0.08])
    folium.map.Marker(centerLl + offset, icon=folium.features.DivIcon(icon_size=(250,36), icon_anchor=(0,0),html=f'<div style="font-size: 12pt">{cityNames.get(cluster)}</div>')
    ).add_to(cityNameLayer)

folium.LayerControl().add_to(map)

map.save(f"../../out/maps/DetectCitiesWithName.html")

In [ ]:
seuil = 100

G = nx.Graph()

for cluster in clusters:
    G.add_node(cluster, pos=cityCenters.get(cluster))

pos = nx.get_node_attributes(G, 'pos')  
for node1 in clusters:
    for node2 in clusters:
        if(node1 > node2):
            loc1 = np.array(cityCenters.get(node1))
            loc2 = np.array(cityCenters.get(node2))
            dist = np.linalg.norm(loc2-loc1)/1000
            weight = (np.min([cityBsStationCount.get(node1), cityBsStationCount.get(node2)]))/(dist)
            if(weight>=0.1):
                G.add_edge(node1, node2, weight=weight)

node_sizes = [cityBsStationCount.loc[node] for node in G.nodes]

G = angle_criterion(G, pos={cluster : transformer.transform(centerXY[0], centerXY[1])[::-1] for (cluster, centerXY) in zip(list(cityCenters.keys()), list(cityCenters.values()))}, min_angle=15)


print(G.edges)
nx.draw(G, pos, node_size = node_sizes)
plt.savefig("../../../weekly_slides/images/road_detection/edges_weight_angle_filtration.png")

In [ ]:
list(cityCenters.values())

In [ ]:
nodes_weights = np.reshape([(cityNames.get(u), w) for u, w in G.nodes(data='weight')], (-1,2))
nodes_weights

In [ ]:
list(cityNames.keys())[list(cityNames.values()).index('Lyon')]

In [ ]:
cityNames.get(6)
cityNames.get(9)

In [ ]:
node1 = list(cityNames.keys())[list(cityNames.values()).index('Toulouse')]
node2 = list(cityNames.keys())[list(cityNames.values()).index('Bordeaux')]
loc1, loc2 = df_extracted[['x','y']].loc[[node1, node2]].values
loc1 = np.array(loc1)
loc2 = np.array(loc2)

dist = np.linalg.norm(loc2-loc1)/1000
print(dist)

print(np.min([cityBsStationCount.get(node1), cityBsStationCount.get(node2)]))
weight = (np.min([cityBsStationCount.get(node1), cityBsStationCount.get(node2)])**4)/(dist**3)
print(weight)